In [1]:
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from gensim.models import KeyedVectors
import re, pickle

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.covid_hack
tweets_db = db.tweets

tweets_db.find_one({'hashtag_id': '#şüpheduymuyorum'})

{'_id': ObjectId('5e8f04f3411c489a86a6e7a2'),
 'created_at': 'Thu Apr 09 10:10:40 +0000 2020',
 'id': 1248191593850568709,
 'id_str': '1248191593850568709',
 'full_text': 'RT @erdmozvrn: Şu Maaşım\nŞuda Kartım Yavrum..\n\n#Şüpheduymuyorum https://t.co/4hlD0Yc4yC',
 'truncated': False,
 'display_text_range': [0, 87],
 'entities': {'hashtags': [{'text': 'Şüpheduymuyorum', 'indices': [47, 63]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'erdmozvrn',
    'name': 'Erdem Özveren',
    'id': 2365427365,
    'id_str': '2365427365',
    'indices': [3, 13]}],
  'urls': [],
  'media': [{'id': 1245763183685570560,
    'id_str': '1245763183685570560',
    'indices': [64, 87],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1245763183685570560/pu/img/qyAgAd7g7ALW-ekf.jpg',
    'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1245763183685570560/pu/img/qyAgAd7g7ALW-ekf.jpg',
    'url': 'https://t.co/4hlD0Yc4yC',
    'display_url': 'pic.twitter.com/4hlD0Yc4yC',
    'ex

In [3]:
word_vectors = KeyedVectors.load_word2vec_format('/home/kemal/Desktop/Workspace/Turkish-Word2Vec/trmodel_me', binary=True)
word_vectors.most_similar(positive=["kral","kadın"],negative=["erkek"])

[('kraliçe', 0.5161710977554321),
 ('kralı', 0.509638786315918),
 ('kralın', 0.4274510443210602),
 ('prens', 0.4047955274581909),
 ('hükümdar', 0.3927798867225647),
 ('kraliçesi', 0.38187533617019653),
 ('kralların', 0.37345683574676514),
 ('veliaht', 0.36857548356056213),
 ('taht', 0.36812296509742737),
 ('krala', 0.3645210266113281)]

In [4]:
word_vectors.get_vector('kadın')
word_vectors.most_similar(positive=["ankara","yunanistan"],negative=["türkiye"])
word_vectors.most_similar(positive=["covid19", "hastalık"])

[('enfeksiyon', 0.8256994485855103),
 ('menenjit', 0.799468994140625),
 ('tüberküloz', 0.7962405681610107),
 ('enfeksiyonu', 0.7794756293296814),
 ('pnömoni', 0.7774212956428528),
 ('enfeksiyonlar', 0.7717976570129395),
 ('frengi', 0.770817756652832),
 ('hastalığın', 0.7696307897567749),
 ('epilepsi', 0.7678350806236267),
 ('hastalığının', 0.7668572664260864)]

In [5]:
def clear_tweet_text(tweet_text):
    tweet_text = re.sub(r'http\S+', '', tweet_text)
    tweet_text = re.sub(r'#\S+', '', tweet_text)
    tweet_text = re.sub(r'@\S+', '', tweet_text)
    tweet_text = re.sub(r'[^\w\s]',' ',tweet_text)
    return tweet_text

def lower_tr(text):
    lowerMap = {ord(u'A'): u'a',ord(u'A'): u'a',ord(u'B'): u'b',ord(u'C'): u'c',ord(u'Ç'): u'ç',ord(u'D'): u'd',ord(u'E'): u'e',ord(u'F'): u'f',ord(u'G'): u'g',ord(u'Ğ'): u'ğ',ord(u'H'): u'h',ord(u'I'): u'ı',ord(u'İ'): u'i',ord(u'J'): u'j',ord(u'K'): u'k',ord(u'L'): u'l',ord(u'M'): u'm',ord(u'N'): u'n',ord(u'O'): u'o',ord(u'Ö'): u'ö',ord(u'P'): u'p',ord(u'R'): u'r',ord(u'S'): u's',ord(u'Ş'): u'ş',ord(u'T'): u't',ord(u'U'): u'u',ord(u'Ü'): u'ü',ord(u'V'): u'v',ord(u'Y'): u'y',ord(u'Z'): u'z',ord(u'W'): u'w',ord(u'Q'): u'q',ord(u'X'): u'x'}
    text = text.translate(lowerMap)
    return text

stop_words = []
with open('stop_words') as f:
    stop_words = f.read().split('\n')

In [11]:
count_fields = {
    'text' : 0,
    'full_text' : 0,
    'normalized_text' : 0,
    'retweeted_status': 0,
    'is_rt': 0,
    'is_retweeted_field' : 0,
    'truncated': 0,
    'quoted_status': 0,
    'tweet_len': 0,
    'word_number': 0,
    'word_number_vec': 0,
    'word_number_not_in_vec': 0,
    'tweet_len_cleared': 0,
    'word_number_cleared': 0,
    'hashtag_number' : 0,
    'url_number' : 0,
    'mention_number' : 0,
    'symbol_number' : 0
}

hashtag_dict = {}

words_not_in_vec = {}

tweet_vectors_all = []

vocabulary = {}

In [12]:
counter = 0
for tweet in tweets_db.find({'hashtag_id': '#şüpheduymuyorum'}):
    counter += 1
    
    for key in count_fields.keys():
        if key in tweet:
            count_fields[key] += 1
    
    if tweet['full_text'].startswith('RT'):
        count_fields['is_rt'] += 1
    else:
        # Not a RT
        count_fields['tweet_len'] += len(tweet['full_text'])
        count_fields['word_number'] += len(tweet['full_text'].split())
        count_fields['hashtag_number'] += len(tweet['entities']['hashtags'])
        count_fields['url_number'] += len(tweet['entities']['urls'])
        count_fields['mention_number'] += len(tweet['entities']['user_mentions'])
        count_fields['symbol_number'] += len(tweet['entities']['symbols'])
        
        for hashtag in tweet['entities']['hashtags']:
            hashtag_text = hashtag['text'].lower()
            if hashtag_text in hashtag_dict:
                hashtag_dict[hashtag_text] += 1
            else:
                hashtag_dict[hashtag_text] = 1
        
        #tweet_text_cleared = clear_tweet_text(tweet['normalized_text'])
        tweet_text_cleared = tweet['zemberek_nlp']['lemmatized_text']
        
        count_fields['tweet_len_cleared'] += len(tweet_text_cleared)
        count_fields['word_number_cleared'] += len(tweet_text_cleared.split())
        
        # Lowercase chars
        tweet_text_cleared = lower_tr(tweet_text_cleared)
        tweet_text_cleared = tweet_text_cleared.lower()
        
        tweet_vectors = []
        
        words_in_vec = 0
        words_not_vec = 0
        # Find out the portion of the words that exists in word vec dictionary
        for word in tweet_text_cleared.split():
            if len(word.strip()) <= 2 or word in stop_words:
                continue
            if word in word_vectors:
                tweet_vectors.append(word_vectors.get_vector(word))
                words_in_vec += 1
                
                if word in vocabulary:
                    vocabulary[word] += 1
                else:
                    vocabulary[word] = 1
            else:
                found_in_vec = False
                remove_char = 7
                for i in range(1,remove_char):
                    if (len(word) - i) < 5:
                        break
                    new_word = word[0:len(word)-i]
                    if new_word in word_vectors:
                        words_in_vec += 1
                        tweet_vectors.append(word_vectors.get_vector(new_word))
                        found_in_vec = True
                        
                        if new_word in vocabulary:
                            vocabulary[new_word] += 1
                        else:
                            vocabulary[new_word] = 1
                    
                        break
                if not found_in_vec:
                    words_not_vec += 1
                    if word in words_not_in_vec:
                        words_not_in_vec[word] += 1
                    else:
                        words_not_in_vec[word] = 1
        
        count_fields['word_number_vec'] += words_in_vec
        count_fields['word_number_not_in_vec'] += words_not_vec
        
        tweet_vectors_all.append(tweet_vectors)
            
        
    
    if tweet['retweeted']:
        count_fields['is_retweeted_field'] += 1
    
    

coverage_ratio = ( count_fields['word_number_vec'] / count_fields['word_number_cleared']) * 100
uncoverage_ratio = ( count_fields['word_number_not_in_vec'] / count_fields['word_number_cleared']) * 100

count_fields['tweet_len'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['word_number'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['tweet_len_cleared'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['word_number_cleared'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['word_number_vec'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['hashtag_number'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['url_number'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['mention_number'] /= (count_fields['full_text'] - count_fields['is_rt'])
count_fields['symbol_number'] /= (count_fields['full_text'] - count_fields['is_rt'])

In [13]:
print(count_fields)

print("Coverage Ratio: {}".format(coverage_ratio))
print("Uncoverage Ratio: {}".format(uncoverage_ratio))

# Most common hastags
hashtag_list = list(hashtag_dict.keys())
hashtag_list.sort(key= lambda item: -1 * hashtag_dict[item])

print("\nHashtags")
for item in hashtag_list[0:10]:
    print(item, hashtag_dict[item])

# Words that do no exist in the word vec
word_list = list(words_not_in_vec.keys())
word_list.sort(key=lambda item: -1 * words_not_in_vec[item])

print("\nWords not in vec")
for item in word_list[0:1000]:
    print(repr(item), words_not_in_vec[item], sep=" - ", end="|")

{'text': 0, 'full_text': 27550, 'normalized_text': 7785, 'retweeted_status': 19774, 'is_rt': 19765, 'is_retweeted_field': 0, 'truncated': 27550, 'quoted_status': 249, 'tweet_len': 161.0134874759152, 'word_number': 18.5514450867052, 'word_number_vec': 10.870905587668593, 'word_number_not_in_vec': 1076, 'tweet_len_cleared': 94.17662170841362, 'word_number_cleared': 15.936544637122672, 'hashtag_number': 1.6213230571612074, 'url_number': 0.05998715478484264, 'mention_number': 0.6879897238278742, 'symbol_number': 0.0}
Coverage Ratio: 68.21369271194364
Uncoverage Ratio: 0.8672803185401319

Hashtags
şüpheduymuyorum 7574
vaka18135 555
gülistan89gündürbulunmuyor 235
suepheduymuyorum 198
lacasadepapel4 188
patoloji2500kadro 174
kitlervarlaborantlarnerede 130
testvarpersonelyok3000labtekatama 122
sözleşmelimemurmağdur 109
covid2019tr 108

Words not in vec
'işkillenmek' - 97|'imek' - 55|'vebal' - 42|'sürçmek' - 26|"chp'li" - 26|'haketmek' - 25|'tabelâ' - 17|'1-2' - 15|'kanalistanbul' - 13|'%80' - 

In [ ]:
stop_words = []
with open('stop_words') as f:
    stop_words = f.read().split('\n')

In [64]:
words_in_vocabulary = list(vocabulary.keys())
words_in_vocabulary.sort(key=lambda word: -1 * vocabulary[word])

wordcloud = {}
for word in words_in_vocabulary:
    if word not in stop_words:
        print(f"{word} : {vocabulary[word]}")
        wordcloud[word] = vocabulary[word]

fatih : 764
bey : 529
şüphe : 511
yok : 493
sağlık : 468
lütfen : 407
istiyoruz : 348
yardım : 325
sayın : 311
sokağa : 306
devlet : 292
çıkma : 276
teknikerleri : 273
portakal : 268
iyi : 241
duyuyorum : 237
gülistan : 231
evde : 225
gün : 224
yasağı : 222
kadro : 215
patoloji : 210
sadece : 205
atama : 201
virüs : 199
zor : 193
bugün : 190
olacak : 188
destek : 187
para : 183
sesimizi : 180
kişi : 178
biyomedikal : 177
zaman : 176
allah : 174
insanlar : 171
tek : 171
olduğundan : 170
türkiye : 168
diş : 166
çalışan : 164
ağız : 161
maalesef : 158
demek : 150
belediye : 150
çalışanları : 149
cumhurbaşkanı : 147
corona : 146
insan : 143
vaka : 142
sağlığı : 142
istanbul : 140
haber : 136
akp : 134
son : 133
halk : 132
işsiz : 132
kredi : 130
ihtiyaç : 130
özel : 126
aynı : 124
den : 122
duymuyorum : 120
isterdim : 120
sayısı : 115
rehabilitasyon : 115
devam : 112
korona : 107
dünya : 106
bağış : 106
abi : 106
chp : 106
istemiyoruz : 105
istiyor : 105
alım : 104
sosyal : 104
büyük : 103

partiler : 7
nice : 7
hakeden : 7
dedirten : 7
gitmiyor : 7
kuranı : 7
deki : 7
samimiyetinden : 7
olmanın : 7
ismet : 7
insandan : 7
yandaşlar : 7
gaz : 7
elinde : 7
basit : 7
olayı : 7
kendilerini : 7
başkanlık : 7
kaygı : 7
yazın : 7
aracılığıyla : 7
doktorun : 7
iman : 7
vatana : 7
günlerinde : 7
seviyede : 7
sıkıntıları : 7
çıkış : 7
özü : 7
yaşa : 7
aldık : 7
alınmalı : 7
adamı : 7
izlediğiniz : 7
söylüyoruz : 7
bugünkü : 7
demeden : 7
veremez : 7
ülkedeki : 7
partilerin : 7
boşta : 7
olayım : 7
sırtı : 7
bilinçli : 7
niyetinden : 7
açıklamasını : 7
suç : 7
taburcu : 7
söylenen : 7
üniversiteler : 7
sıkıntı : 7
ettiler : 7
duvar : 7
sol : 7
elde : 7
evime : 7
cesur : 7
mağduriyet : 7
listede : 7
geride : 7
pahasına : 7
erkek : 7
denen : 7
suçu : 7
çıkmasın : 7
bilime : 7
inanmak : 7
almalı : 7
bakınca : 7
batman : 7
soran : 7
arasına : 7
çıkarıp : 7
diyelim : 7
söyleyen : 7
diyeceğim : 7
kendimi : 7
mhp : 7
akparti : 7
makam : 7
doğruluğundan : 7
şartları : 7
dili : 7
ini : 7
kul

tabip : 4
zihniyetle : 4
dönük : 4
bozuktu : 4
param : 4
defol : 4
şişli : 4
geçtiğimiz : 4
sürecin : 4
demesin : 4
ettiklerine : 4
maden : 4
sahil : 4
soruna : 4
izliyor : 4
ulaştır : 4
söylediklerinde : 4
olduğumuzun : 4
gelmediği : 4
ısrarları : 4
duymayan : 4
kararını : 4
döneceklerini : 4
bilmenizi : 4
kanserden : 4
gelecekten : 4
binlere : 4
artacağından : 4
gönderi : 4
yaşındayım : 4
siyasilerin : 4
ayar : 3
tedavide : 3
unsuru : 3
farkını : 3
iyilik : 3
cennet : 3
sms : 3
memleketim : 3
yorgun : 3
cemaat : 3
anlatsın : 3
tırnağı : 3
durumunu : 3
tecavüz : 3
ödül : 3
bakanını : 3
kurumda : 3
konuşma : 3
çeviriyor : 3
üretimine : 3
özelde : 3
anlayacak : 3
uyumu : 3
vermemek : 3
gitmiş : 3
gittikçe : 3
yoldaş : 3
savaşıyor : 3
çıkarın : 3
kokusu : 3
başlayacak : 3
yalanı : 3
dengeler : 3
gelene : 3
hastayı : 3
servise : 3
eşlik : 3
adaletsizlik : 3
haberle : 3
cezasını : 3
yazmış : 3
atmış : 3
durun : 3
kıvırma : 3
lübnan : 3
bölgelerinden : 3
isyan : 3
ölmektense : 3
yalanları :

olmamızın : 2
bulandı : 2
sözleri : 2
alakasız : 2
moralini : 2
yazdı : 2
halkıyla : 2
kocasından : 2
samimiyetsiz : 2
utanmaz : 2
sözcü : 2
böylesi : 2
duymayacak : 2
mantık : 2
seviyor : 2
layıkıyla : 2
yapmaktadır : 2
görevin : 2
alkışlı : 2
kürek : 2
çoban : 2
toz : 2
duman : 2
gerekenleri : 2
geldiyse : 2
dedikten : 2
içerisindeki : 2
fbı : 2
yaşat : 2
malların : 2
rahman : 2
rahim : 2
hayatımda : 2
ulaştırılmak : 2
yardımla : 2
kalbimiz : 2
uğraşıyoruz : 2
yaradanı : 2
bahis : 2
zengine : 2
sırtını : 2
aysu : 2
anlatmış : 2
abone : 2
ilave : 2
silik : 2
anlamıyorsun : 2
mala : 2
sahibine : 2
kek : 2
yapmıştır : 2
ocak : 2
edebilirim : 2
ınstagram : 2
instagram : 2
çalıştılar : 2
günlerim : 2
buhar : 2
çıkarsa : 2
bakırköy : 2
umursamadı : 2
alınmamasını : 2
zirvesinde : 2
yatsı : 2
hocanın : 2
uzmanları : 2
rakamdan : 2
düşmek : 2
gönülden : 2
güvenerek : 2
lirayı : 2
gönderin : 2
ihanetten : 2
haberlerle : 2
masraflarını : 2
teşkilatı : 2
oynayın : 2
gösterelim : 2
ruhunu : 2
fe

koymuş : 2
çekem : 2
zenginlerinin : 2
bildirilen : 2
halletmek : 2
bakarsanız : 2
ilçesinde : 2
dağıtılması : 2
ertelemeler : 2
karıştırdığı : 2
ençok : 2
bizimki : 2
başlattığını : 2
yapılırsa : 2
araçları : 2
derecede : 2
edilmesin : 2
edilmez : 2
mühendisliği : 2
imiz : 2
onarımı : 2
vermekte : 2
memleketlerine : 2
böylelikle : 2
kaybedenler : 2
amacına : 2
kredilerinde : 2
neresi : 2
istanbuldaki : 2
yıldızlı : 2
cebinden : 2
oynayan : 2
trafiği : 2
nisana : 2
kesimlerine : 2
yolculukları : 2
tarafsızca : 2
kullanıldığında : 2
merkezde : 2
biriyim : 2
duygusunu : 2
679 : 2
fiyasko : 2
devletinin : 2
sözlerin : 2
tabipler : 2
çalışıyorsunuz : 2
numunelerin : 2
filmi : 2
gördüğümüzde : 2
kaçışı : 2
ilinde : 2
yönetildiği : 2
alırdı : 2
ödeyen : 2
temsilcileri : 2
dosya : 2
kafam : 2
ayırdığı : 2
yiyorum : 2
yapılandır : 2
0212 : 2
414 : 2
9000 : 2
bayraklı : 2
akıyla : 2
istediniz : 2
yönetimlerin : 2
dışından : 2
kavgası : 2
yönet : 2
miktar : 2
eşine : 2
öleceğimi : 2
mahkumlar : 

seyirciyi : 1
büyüğe : 1
unutursun : 1
derdiyle : 1
uğraşanlar : 1
çarpar : 1
hamud : 1
gırtlak : 1
tumba : 1
sulu : 1
oluşturmalarına : 1
beyninin : 1
hizmetinde : 1
finali : 1
geceye : 1
kanka : 1
çekiyorsa : 1
sevgisinden : 1
alkolik : 1
yasaklarına : 1
uysa : 1
incitme : 1
canlıyı : 1
kalpteki : 1
almıyorsa : 1
yaratıyor : 1
lideridir : 1
rasulullah : 1
sallallahu : 1
aleyhi : 1
vesselam : 1
borçtan : 1
düşmanın : 1
gelmesinden : 1
sevindir : 1
sığınır : 1
ibn : 1
habban : 1
istiare : 1
çekilecek : 1
senaryoda : 1
rolünde : 1
olsaydın : 1
özledin : 1
görünenler : 1
dininin : 1
dururlar : 1
semavi : 1
dinlerin : 1
hepsinde : 1
yıkılmaz : 1
bilincinde : 1
toprağının : 1
tadını : 1
davrandı : 1
çevremizi : 1
başsız : 1
kediyi : 1
saniyede : 1
kopya : 1
dikkatini : 1
çevirmiş : 1
büyülenmiş : 1
çapta : 1
gizlenmeye : 1
dikkatimizi : 1
toplaya : 1
köpeği : 1
birazdan : 1
atmaya : 1
kirin : 1
yıkanma : 1
saymayı : 1
pislikten : 1
saymaktan : 1
olumsuzluğu : 1
vadeden : 1
ölmemek : 1
şuru

sunacağı : 1
teröristler : 1
kulaklar : 1
kalınacak : 1
kaçalım : 1
bulgurdan : 1
röportajdan : 1
alıntı : 1
iktisatçının : 1
gittiğinde : 1
bütündür : 1
ayrılamaz : 1
sokulmuş : 1
olabileceğin : 1
kfar : 1
münferit : 1
konuşmadı : 1
bulmalı : 1
hapsinde : 1
alınanlar : 1
garson : 1
komiler : 1
kahramanmaraş : 1
aralarındaki : 1
azalır : 1
azalmaya : 1
milyona : 1
yetkililerinin : 1
öfkeli : 1
güvenmez : 1
kaşına : 1
ayrılığı : 1
doğdu : 1
ilimi : 1
başkanıdır : 1
konuşmaları : 1
uçuşur : 1
koyulmasında : 1
bilmeyi : 1
hayatımdan : 1
partilerle : 1
yüreğin : 1
yayına : 1
istersen : 1
çalıştır : 1
konuşamıyor : 1
devletinde : 1
gereğinden : 1
yayınevi : 1
satışlarının : 1
düşmesi : 1
kazanımı : 1
satışları : 1
yayınlarını : 1
buruna : 1
fonundan : 1
çalacakları : 1
atlatıldıktan : 1
sorgulayıp : 1
değişikliklere : 1
alabilirler : 1
isyanını : 1
yansıtmış : 1
gitmesine : 1
sesine : 1
okuyabilirsiniz : 1
kaldırdığını : 1
gösterirse : 1
vergiyi : 1
yapıştırır : 1
birleşmeyi : 1
sinde : 1
b

solun : 1
gidilirken : 1
duyulacağını : 1
çılgınlığı : 1
izolasyonun : 1
gerçeklemesi : 1
game : 1
over : 1
görmüş : 1
düşmanlarım : 1
masur : 1
göçmüş : 1
arasından : 1
getirilirse : 1
yasaklanır : 1
düşecektir : 1
tetik : 1
laboratuvarıdır : 1
yaparsan : 1
çaresi : 1
dışarda : 1
mikrofona : 1
lunapark : 1
inanacak : 1
sayacını : 1
okuyamadı : 1
sayacı : 1
bildirin : 1
önlemlerden : 1
haftasonu : 1
altındadır : 1
şirketlerinin : 1
modeline : 1
geçmesini : 1
başvuran : 1
aramızda : 1
tanede : 1
olabilen : 1
bölüştürme : 1
zorlaştı : 1
hayatımıza : 1
mantığında : 1
davası : 1
hukuki : 1
ersan : 1
şen : 1
yöntemlerle : 1
gösterilmesidir : 1
iyiliği : 1
emreden : 1
kötülükten : 1
men : 1
topluluk : 1
kurtuluşa : 1
erenler : 1
onlardır : 1
imran : 1
104 : 1
hesabından : 1
toplamadan : 1
çocuklardan : 1
marketlerde : 1
malzemeler : 1
göndermekten : 1
üstüner : 1
yaktılar : 1
ergenekon : 1
hocadan : 1
hijyene : 1
temizliği : 1
mesafeye : 1
personal : 1
space : 1
yığma : 1
zararını : 1
likidi

durak : 1
öldüreceğim : 1
ölürsen : 1
keseceği : 1
getirisini : 1
seslendi : 1
otizmin : 1
dengi : 1
makamlarla : 1
irtibata : 1
geçeceğine : 1
göndermelerde : 1
içereceği : 1
şule : 1
çat : 1
medik : 1
hastanesini : 1
söylemiyorum : 1
çevirmeyi : 1
umuyordu : 1
çevrilebilecek : 1
olmuşlar : 1
kalmamıştır : 1
günümde : 1
engelle : 1
krizin : 1
aldığına : 1
hukuku : 1
saldırmak : 1
sınırlarını : 1
demeçlerde : 1
kararların : 1
dudağın : 1
hastada : 1
kural : 1
delinir : 1
yavaşa : 1
icrada : 1
337 : 1
peşinat : 1
inmiş : 1
turu : 1
ucuzladı : 1
birimizin : 1
dostu : 1
celladı : 1
yakınları : 1
girdiğini : 1
pınar : 1
saip : 1
annemin : 1
kadındır : 1
kapatmadığı : 1
kullandım : 1
oturdum : 1
artmaz : 1
sineye : 1
rampası : 1
üniversitesine : 1
süreceği : 1
elemanlarını : 1
980 : 1
205 : 1
725 : 1
çalışmalarım : 1
anlamaları : 1
anlayacakları : 1
gülümse : 1
araçtan : 1
tatiline : 1
soros : 1
görüşüp : 1
yetkinin : 1
girilecektir : 1
beklemeden : 1
kuraklık : 1
patlatacak : 1
toprağını :

In [65]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 

In [66]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stop_words, 
                min_font_size = 10).generate(wordcloud) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

TypeError: expected string or bytes-like object

In [73]:
with open('tweet_vecs', "wb") as f:
    pickle.dump(tweet_vectors_all, f)

In [75]:
word_vectors.most_similar(positive=['cılar'])

[('yaparsan', 0.6264301538467407),
 ('yazdığımı', 0.6164571046829224),
 ('buyurdum', 0.6104065179824829),
 ('istersiniz', 0.5952123403549194),
 ('umduk', 0.5807696580886841),
 ('то', 0.5804905891418457),
 ('nǐ', 0.5803848505020142),
 ('söylersin', 0.5797668695449829),
 ('gelegenheit', 0.5797350406646729),
 ('чажымда', 0.5746351480484009)]